In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import pyneb as pn

In [ ]:
O3 = pn.Atom('O', 3, NLevels=5)
N2 = pn.Atom('N', 2, NLevels=5)
O2 = pn.Atom('O', 2, NLevels=5)
S2 = pn.Atom('S', 2, NLevels=5)

# ABUNDANCES DERIVATIONS WITH THE DIRECT METHOD

Atom class has the getIonAbundance method to determine ionic abundance from Te, Ne and a line intensity (normalised to Hbeta).

In [ ]:
print(12 + np.log10(O3.getIonAbundance(int_ratio=125., tem=1e4, den=1e2, wave=5007, Hbeta=100.0)))

Let's see the effects of the line intensity, Te and Ne on the determined abundances

In [ ]:
def print_abO(tem = 1e4, den = 1e2, int_ratio = 125., Hbeta = 100.):
    print('Te = {:6.0f}, Ne = {:6.0f}, [OIII]/Hb = {:5.1f} -> 12 + log(O++/H+) = {:.2f}'.format(tem, den, int_ratio/Hbeta, 
                                                                      12 + np.log10(O3.getIonAbundance(int_ratio=int_ratio, 
                                                                                                       tem=tem, den=den, 
                                                                                                       wave=5007, 
                                                                                                       Hbeta=Hbeta))))
print_abO(tem = 1e4, den = 1e2, int_ratio = 125.)
print_abO(tem = 1e4, den = 1e2, int_ratio = 155.)
print_abO(tem = 1.3e4, den = 1e2, int_ratio = 125.)
print_abO(tem = 1e4, den = 1e5, int_ratio = 125.)

## Let's make the process more efficient

Te and Ne are determined, for different regions of the nebula, corresponding to different ions identified by their ionization potentials. It is now possible to determine the ionic abundances that corresponds to each observed emission line.

In [ ]:
# Fill the following with values determined in the previous notebook (or whatever value you want)
dens = 4600
T_N2 = 12800.
T_O3 = 11927. 

In [ ]:
%%writefile observations2.dat
LINE   MYOBJECT errMYOBJECT
S4_10.5m   7.00  0.25
Ne2_12.8m  8.30  0.25
Ne3_15.6m 34.10  0.20
S3_18.7m  10.00  0.20
O2_3726A  39.70  0.05
O2_3729A  18.60  0.05
Ne3_3869A 18.90  0.05
Ne3_3968A  6.4   0.10
S2_4069A   0.85  0.15
S2_4076A   0.45  0.15
O3_4363A   4.36  0.10
H1r_4861A 100.0  0.00
O3_5007A 435.09  0.05
N2_5755A   0.51  0.15
S3_6312A   0.76  0.15
O1_6300A   1.69  0.15
O1_6364A   0.54  0.15
N2_6548A   6.84  0.10
H1r_6563A 345.0  0.05
N2_6584A  19.00  0.10
S2_6716A   1.22  0.15
S2_6731A   2.18  0.15
Ar3_7136A  4.91  0.15
O2_7319A+  6.54  0.10
O2_7330A+  5.17  0.10

In [ ]:
obs = pn.Observation('observations1.dat', fileFormat='lines_in_rows_err_cols', corrected=False) 
obs.extinction.law = 'CCM89'  # define the extinction law from Cardelli et al.
obs.def_EBV(label1="H1r_6563A", label2="H1r_4861A", r_theo=2.85) # Compute the redenning correction
obs.correctData(normWave=4861)                      # Correct the data from attenuation

In [ ]:
obs.getUniqueAtoms()

In [ ]:
all_atoms = pn.getAtomDict(atom_list=obs.getUniqueAtoms(), NLevels=5)

In [ ]:
all_atoms

In [ ]:
# define a dictionary for the ionic abundances
ab_dic = {}
# we  use the following lines to determine the ionic abundances
ab_labels = ['N2_6584A', 'N3_57.4m', 'N4_1487A', 
             'O2_3726A', 'O2_3729A', 'O3_5007A', 'O3_4363A', 'O4_25.9m',
             'Ne2_12.8m', 'Ne3_3869A', 'Ne3_3968A', 'Ne3_15.6m', 'Ne5_3426A', 'Ne6_7.6m',
             'S2_4069A', 'S2_4076A', 'S2_6716A', 'S2_6731A', 'S3_6312A',
             'Ar3_7136A']
# loop on the observed lines to determine the corresponding ionic abundances
for line in obs.getSortedLines():
    # this is one way to define temp and dens in each zone
    # must be adapted to each case
    if (line.atom in all_atoms) and (line.label in ab_labels):
        IP_cut = 30. # The IPs have been imported before
        if all_atoms[line.atom].IP > IP_cut:
            temp = T_O3
            IP_used = 'H'
        else:
            temp = T_N2
            IP_used = 'L'                 
        ab = all_atoms[line.atom].getIonAbundance(line.corrIntens, temp, dens, 
                                                  to_eval=line.to_eval, 
                                                  Hbeta=100)[0]
        print('{0:10s}  {1}   {2:6.3f}'.format(line.label, IP_used, ab * 1e6))
        # More than one abundance per ion can be obtained:
        if line.atom not in ab_dic:
            ab_dic[line.atom] = []
        ab_dic[line.atom].append(ab)

What is the reason for abundances to differ for a given ion?

How could you "make" all the abundances for a given ion to agree? Is this the right way? ;-)

In [ ]:
ab_dic

How will you define the "adopted" abundance for each ion?

In [ ]:
for atom in ab_dic:
    mean = np.mean(np.asarray(ab_dic[atom]))
    ab_dic[atom] = mean # It may NOT be the best way
ab_dic

## Compute the total abundances from the ionic abundances

$$ \frac{X}{H} = \sum_i \frac{X^i}{H^+} $$

If all the ions are observed, the situation is simple.

This is not always the case, for example $N^{++}$ has no observable line in the visible.

Considere that the IPs of N and O are close:

In [ ]:
pn.print_IPs(N_elems=8, N_ions=2)

One can pretend that the ionization of both atoms are also similar: $N^+/N = O^+/O$. 

This leads to: $$ \frac{N}{H} = \frac{N^+}{H^+} . \frac{N}{N^+} . \frac{H^+}{H} = \frac{N^+}{H^+} . \frac{O}{O^+} = \frac{N^+}{H^+} . ICF(N^+) $$

The ICF is the coefficient by which one ionic abundance (or the sum of the observed ionic abundances) of an element is multiplied to obtain the total (elemental) abundance.

In [ ]:
# Instantiation of the ICF object
icf = pn.ICF()

In [ ]:
# The name of the ICF is related to the paper and the equation in the paper. It is used to obtain an elemental abundance, from the sum of some ionic abundances.
icf.printAllICFs()

In [ ]:
# ICFs are from papers:
for type_ in icf.all_icf_refs:
    print(type_, icf.all_icf_refs[type_])

In [ ]:
# And they apply to different regions: HII, PNe, both.
icf.getAvailableICFs(type_= ('HII', 'All'))

In [ ]:
# Some ICFs requiere the He ionic fracions. In lack of HeII line, we put He++ abundance to a very low value
ab_dic['He1r'] = 0.1
ab_dic['He2r'] = 0.00001
# Computing the elemental abundances from all ICF rules.

# The following computes the elemental abundances from the rules.

def pretty_print_icf(icf=None, icf_family='direct'):
    if icf is None:
        icf = pn.ICF()
    elem_abun = icf.getElemAbundance(ab_dic, icf_family=icf_family)
    printRule = True
    for icf_ref in np.sort(list(elem_abun.keys())):
        if np.isfinite(elem_abun[icf_ref]) and np.log10(elem_abun[icf_ref]).all() > -10:
            if printRule:
                print('{0:12s} {1:2s} = {3:5.2f} using {2}'.format(icf_ref, 
                                                  icf.all_icfs[icf_ref]['elem'],
                                                  icf.getExpression(icf_ref), 
                                                  np.log10(elem_abun[icf_ref])+12))
            else:
                print('{0:12s} {1}={2:.3}'.format(icf_ref, 
                                                  icf.all_icfs[icf_ref]['elem'],
                                                  np.log10(elem_abun[icf_ref])+12))


In [ ]:
pretty_print_icf()

In [ ]:
pretty_print_icf(icf, 'PHCD07')

In [ ]:
pretty_print_icf(icf, 'Ial06')

### !!! the following ones are for PNe !!!

In [ ]:
pretty_print_icf(icf, 'KB94')

## Monte Carlo

In [ ]:
N_MC = 2000
#pn.config.use_multiprocs() # This does not work well from within a Notebooks. But in scripts from termiinal it seems OK.
obs = pn.Observation('observations1.dat', fileFormat='lines_in_rows_err_cols', corrected=False)
obs.addMonteCarloObs(N_MC)
obs.extinction.law = 'CCM89'  # define the extinction law from Cardelli et al.
obs.def_EBV(label1="H1r_6563A", label2="H1r_4861A", r_theo=2.85) # Compute the redenning correction
obs.correctData(normWave=4861)

In [ ]:
obs.lineLabels

In [ ]:
obs.extinction.cHbeta[0:10]

In [ ]:
f, ax = plt.subplots()
ax.hist(obs.getIntens(returnObs=True)['O3_5007A'], bins=np.linspace(360, 500, 100), label='Observed');
ax.hist(obs.getIntens()['O3_5007A'], bins=np.linspace(360, 500, 100), alpha=0.5,label='Deredenned')
ax.legend();

In [ ]:
diags = pn.Diagnostics()
diags.addDiagsFromObs(obs)
pn.log_.timer('Start', quiet=True, calling='Computing Te Ne')
T_O3, dens_S2 = diags.getCrossTemDen(diag_tem='[OIII] 4363/5007', diag_den='[SII] 6731/6716', obs=obs)
T_N2, dens_O2 = diags.getCrossTemDen(diag_tem='[NII] 5755/6584', diag_den='[OII] 3726/3729', obs=obs)
pn.log_.timer('End')

In [ ]:
f, ax = plt.subplots()
ax.hist(T_O3, bins=100);

## A very experimental way to determine Te and Ne is using a Neural Network. It is still in development, but very promising. Contact me if you have huge number of data to deal with.

In [ ]:
try:
    from ai4neb import manage_RM
except:
    !pip install -U git+https://github.com/morisset/AI4neb.git
    from ai4neb import manage_RM

In [ ]:
pn.log_.timer('Start', quiet=True, calling='Computing Te Ne with ANN')
T_O3ANN, dens_S2ANN = diags.getCrossTemDen(diag_tem='[OIII] 4363/5007', diag_den='[SII] 6731/6716', obs=obs, 
                                           use_ANN=True, limit_res=True, start_tem=3e3, end_den=1e6)
T_N2ANN, dens_O2ANN = diags.getCrossTemDen(diag_tem='[NII] 5755/6584', diag_den='[OII] 3726/3729', obs=obs, 
                                           use_ANN=True, limit_res=True, start_tem=3e3, end_den=1e6)
pn.log_.timer('End')

In [ ]:
f, ax = plt.subplots()
ax.hist(T_O3, bins=100, label='direct')
ax.hist(T_O3ANN, bins=100, alpha=0.3, label='ANN')
ax.legend();

In [ ]:
f, ax = plt.subplots()
ax.hist(T_N2, bins=100, label='direct')
ax.hist(T_N2ANN, bins=100, alpha=0.3, label='ANN')
ax.legend();

In [ ]:
print('T[NII]      = {:.0f} +/- {:.0f}'.format(np.nanmedian(T_N2), np.nanstd(T_N2)))
print('T[NII]  ANN = {:.0f} +/- {:.0f}'.format(np.nanmedian(T_N2ANN), np.nanstd(T_N2ANN)))
print('T[OIII]     = {:.0f} +/- {:.0f}'.format(np.nanmedian(T_O3), np.nanstd(T_O3)))
print('T[OIII] ANN = {:.0f} +/- {:.0f}'.format(np.nanmedian(T_O3ANN), np.nanstd(T_O3ANN)))

In [ ]:
# define a dictionary for the ionic abundances
ab_dic = {}
# we  use the following lines to determine the ionic abundances
ab_labels = ['N2_6584A', 'N3_57.4m', 'N4_1487A', 
             'O2_3726A', 'O2_3729A', 'O3_5007A', 'O4_25.9m',
             'Ne2_12.8m', 'Ne3_3869A', 
             'S2_6716A', 'S2_6731A','S3_6312A',
             'Ar3_7136A']
# loop on the observed lines to determine the corresponding ionic abundances
for line in obs.getSortedLines():
    # this is one way to define temp and dens in each zone
    # must be adapted to each case
    if (line.atom in all_atoms) and (line.label in ab_labels):
        IP_cut = 30. # The IPs have been imported before
        if all_atoms[line.atom].IP > IP_cut:
            temp = T_O3ANN
            IP_used = 'H'
        else:
            temp = T_N2ANN
            IP_used = 'L'                 
        ab = all_atoms[line.atom].getIonAbundance(line.corrIntens, temp, dens_S2, 
                                                  to_eval=line.to_eval, 
                                                  Hbeta=100)
        # More than one abundance per ion can be obtained:
        if line.atom not in ab_dic:
            ab_dic[line.atom] = []
        ab_dic[line.atom].append(ab)

In [ ]:
len(ab_dic['S2'])

In [ ]:
np.nanmedian(np.log10(ab_dic['S2']), axis=1)

In [ ]:
for ion in ab_dic:
    ab_dic[ion] = np.nanmedian(ab_dic[ion], axis=0)

In [ ]:
ab_dic['He1r'] = 0.1 * np.ones_like(ab_dic['O2']) + np.random.randn(N_MC+1) * 0.005
ab_dic['He2r'] = 0.00001 * np.ones_like(ab_dic['O2'])

In [ ]:
icf = pn.ICF()

In [ ]:
elem_abun = icf.getElemAbundance(ab_dic, icf_family='Ial06', use_MC=True)

In [ ]:
from pyneb.utils.misc import get_reduced_dic

In [ ]:
elem_abun2 = get_reduced_dic(N_MC, obs.n_obs_origin, elem_abun, error_method='quants', abund12=True)    

In [ ]:
for icf_ref in np.sort(list(elem_abun2.keys())):
    if icf_ref[-1] != 'e':
        if np.log10(elem_abun2[icf_ref]) > -10:
            value = elem_abun2[icf_ref][0]
            err = elem_abun2[icf_ref+'_e'][0]
            print('{:10s} {:2s} = {:4.2f} +/- {:4.2f} using {}'.format(icf_ref, 
                                              icf.all_icfs[icf_ref]['elem'],
                                                value,
                                                err      ,    
                                              icf.getExpression(icf_ref), 
                                              ))
